**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:** Augusto Cesar Simas dos Reis\
**Matricula:** 2139598

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?
    - **O algoritmo de Dijkstra supõe que, ao visitar um nó, o caminho encontrado até ele é o mais curto possível naquele momento.
Essa suposição só é verdadeira se todas as arestas tiverem peso não negativo.**

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?
    - **O algoritmo de Dijkstra, quando implementado usando uma lista de adjacência e uma heap (heapq) como fila de prioridade, apresenta uma complexidade eficiente para grafos grandes.**\
💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [22]:
# Insira seu código aqui
import heapq

def dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    parent = {node: None for node in graph}
    dist[start] = 0

    pq = [(0, start)]

    while pq:
        current_dist, current_node = heapq.heappop(pq)

        if current_dist > dist[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_dist + weight

            if distance < dist[neighbor]:
                dist[neighbor] = distance
                parent[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    return dist, parent

def reconstruct_path(parent, target):
      path = []
      current = target

      while current is not None:
          path.append(current)
          current = parent[current]

      path.reverse()
      return path

grafo = {
    "Hospital": {"A": 1, "C": 9},
    "A": {"Hospital": 3, "B": 2, "C": 4},
    "B": {"A": 2, "Local": 3},
    "C": {"Hospital":1, "A": 4, "Local": 8},
    "Local": {"B": 3, "C": 1},
}
dist, parent = dijkstra(grafo, "Hospital")

path = reconstruct_path(parent, "Local")

print("Caminho mais rapido:", ", ".join(path))
print("Tempo: ", dist["Local"], "minutos")

Caminho mais rapido: Hospital, A, B, Local
Tempo:  6 minutos


**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?
    - **Se a heurística h(n)h(n) for admissível e consistente, o A* nunca expande mais nós que Dijkstra**

2. Por que a heurística Manhattan é admissível nesse caso?
    - **A heurística de distância Manhattan é considerada admissível porque ela nunca superestima o custo real mínimo necessário para alcançar o objetivo em um ambiente em grade onde o agente só pode se mover nas direções horizontais e verticais, norte, sul, leste e oeste.**

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [24]:
# Insira seu código aqui
import random
import heapq
def gerarGrid(tam=20, obstaculo=0.15):
    grid = []
    for _ in range(tam):
        row = [0 if random.random() > obstaculo else 1 for _ in range(tam)]
        grid.append(row)
    return grid

grid = gerarGrid()
def a_star(grid, start, goal, h):
    open_set = []
    heapq.heappush(open_set, (0 + h(start, goal), 0, start))

    parent = {start: None}
    g_score = {start: 0}

    while open_set:
        f, current_g, current = heapq.heappop(open_set)

        if current == goal:
            path = []
            while current:
                path.append(current)
                current = parent[current]
            path.reverse()
            return path

        x, y = current
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x + dx, y + dy
            neighbor = (nx, ny)

            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 0:
                tentative_g = current_g + 1
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + h(neighbor, goal)
                    heapq.heappush(open_set, (f_score, tentative_g, neighbor))
                    parent[neighbor] = current

    return None



def heuristca(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

start = (0, 0)
goal = (19, 19)

path = a_star(grid, start, goal, heuristca)

if path:
    print("Caminho encontrado: ", len(path))
else:
    print("Nenhum caminho ")



Caminho encontrado:  39


**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?
    - In-ordem: Ordenar dados (em BST)
    - Pré-ordem: Clonar/serializar árvore, processar raiz primeiro
    - Pós-ordem: Liberar memória, calcular totais, processar filhos primeiro

In [25]:
## Insira seu código aqui
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.value:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

values = [50, 30, 70, 20, 40, 60, 80, 35, 45]
root = None

for val in values:
    root = insert(root, val)

def in_order(root):
    if root:
        in_order(root.left)
        print(root.value, end=", ")
        in_order(root.right)

def pre_order(root):
    if root:
        print(root.value, end=", ")
        pre_order(root.left)
        pre_order(root.right)

def post_order(root):
    if root:
        post_order(root.left)
        post_order(root.right)
        print(root.value, end=", ")

print("in_order()" )
in_order(root)
print("\n-------------")
print("pre_order()")
pre_order(root)
print("\n-------------")
print("port_order()")
post_order(root)


in_order()
20, 30, 35, 40, 45, 50, 60, 70, 80, 
-------------
pre_order()
50, 30, 20, 40, 35, 45, 70, 60, 80, 
-------------
port_order()
20, 35, 45, 40, 30, 60, 80, 70, 50, 

**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?
    - Embora o A* seja eficiente ao usar heurísticas informadas, ele pode se tornar desvantajoso quando o espaço de busca é muito grande ou quando a heurística é difícil de calcular ou pouco precisa. Nesses casos, o custo de computar a heurística a cada nó pode superar os ganhos em eficiência. Por exemplo, em um mapa de cidade muito grande com milhares de ruas e interseções, usar A* com uma heurística complexa de tráfego em tempo real pode ser mais lento que um algoritmo de busca cega simples, como BFS em um caminho curto, pois o tempo gasto avaliando heurísticas supera a economia de nós visitados. Além disso, se a heurística subestima muito o custo real, o A* se aproxima do comportamento do Dijkstra, perdendo a vantagem.



2. Diferencie corretude e otimalidade nos algoritmos estudados.
    - Corretude refere-se a um algoritmo sempre produzir uma solução válida quando existe uma; ele nunca gera respostas inválidas. Já a otimalidade significa que o algoritmo encontra a melhor solução possível de acordo com algum critério, como custo mínimo ou caminho mais curto. Por exemplo, o BFS é correto para encontrar caminhos em grafos não ponderados e é ótimo no sentido de encontrar o caminho com menor número de passos. O DFS, por outro lado, é correto se a solução existir, mas não é necessariamente ótimo, pois pode encontrar caminhos mais longos primeiro.


3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.
    - In-ordem: Em uma BST de preços de produtos de um e-commerce, o in-ordem permite listar produtos do mais barato ao mais caro, essencial para filtros de busca por preço.

    - Pré-ordem: Ao salvar ou clonar uma estrutura de árvore sintática em compiladores, o pré-ordem preserva a hierarquia dos nós, permitindo reconstruir exatamente a árvore original.

    - Pós-ordem: Em sistemas de arquivos ou jogos, ao liberar memória de estruturas hierárquicas, é necessário processar primeiro os filhos antes da raiz, evitando erros de referência.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?
    - Heurísticas inconsistentes podem fazer com que o A* reavalie o mesmo nó várias vezes, aumentando o número de expansões e tornando a busca menos eficiente. Em casos extremos, o algoritmo pode não garantir a otimalidade, encontrando caminhos subótimos. Por exemplo, em um jogo de labirinto, se a heurística de distância até a saída subestima em um ponto e superestima logo após, o A* pode seguir caminhos desnecessários e revisitar posições, atrasando a solução.

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

Insira suas respostas aqui

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.